In [31]:
>>> import oamap.source.root
>>> import uproot
>>> events = uproot.open("http://scikit-hep.org/uproot/examples/HZZ.root")["events"].oamap()
events.schema.content.rename("NElectron", "electrons")
events.schema.content["electrons"].content.rename("Electron_Px", "px")
events.schema.content["electrons"].content.rename("Electron_Py", "py")
events.schema.content["electrons"].content.rename("Electron_Pz", "pz")
events.schema.content["electrons"].content.rename("Electron_E", "energy")
events.schema.content["electrons"].content.rename("Electron_Iso", "isolation")
events.schema.content["electrons"].content.rename("Electron_Charge", "charge")
events.schema.content.rename("NMuon", "muons")
events.schema.content["muons"].content.rename("Muon_Px", "px")
events.schema.content["muons"].content.rename("Muon_Py", "py")
events.schema.content["muons"].content.rename("Muon_Pz", "pz")
events.schema.content["muons"].content.rename("Muon_E", "energy")
events.schema.content["muons"].content.rename("Muon_Iso", "isolation")
events.schema.content["muons"].content.rename("Muon_Charge", "charge")
events.schema.content.rename("NPhoton", "photons")
events.schema.content["photons"].content.rename("Photon_Px", "px")
events.schema.content["photons"].content.rename("Photon_Py", "py")
events.schema.content["photons"].content.rename("Photon_Pz", "pz")
events.schema.content["photons"].content.rename("Photon_E", "energy")
events.schema.content["photons"].content.rename("Photon_Iso", "isolation")
events.schema.content.rename("NJet", "jets")
events.schema.content["jets"].content.rename("Jet_Px", "px")
events.schema.content["jets"].content.rename("Jet_Py", "py")
events.schema.content["jets"].content.rename("Jet_Pz", "pz")
events.schema.content["jets"].content.rename("Jet_E", "energy")
events.schema.content["jets"].content.rename("Jet_ID", "id")
events.schema.content["jets"].content.rename("Jet_btag", "btag")
events.regenerate()
>>> import functional
from math import *
def mass(*particles):
    energy = particles.map(lambda particle: particle.energy).sum
    px = particles.map(lambda particle: particle.px).sum
    py = particles.map(lambda particle: particle.py).sum
    pz = particles.map(lambda particle: particle.pz).sum
    return sqrt(energy**2 - px**2 - py**2 - pz**2)

I've found that it would be simpler by first calculating Z mass and then use `pairs` to calculate Higgs mass:

In [57]:
from IPython.display import display, Math, Latex

display(Math(r'm=\sqrt{energy^{2}-p^{2}_{x}-p^{2}_{y}-p^{2}_{z}}'))
display(Math(r'm_{H}=\sqrt{m^2_{z_{1}}+m^2_{z_{2}}}=\sqrt{energy^{2}_1-p^{2}_{x1}-p^{2}_{y1}-p^{2}_{z1}+energy^{2}_2-p^{2}_{x2}-p^{2}_{y2}-p^{2}_{z2}} \\ \\
 = \sqrt{(energy^{2}_1+energy^{2}_2)-(p^{2}_{x1}+p^{2}_{y1}+p^{2}_{z1}+p^{2}_{x2}+p^{2}_{y2}+p^{2}_{z2})}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [94]:
es = events
masses = (es
          .lazy
          .map(lambda event: 
               [event.electrons.pairs(mass), event.muons.pairs(mass)]) # need only electrons and muons
          .map(lambda event: event[0]+event[1]) # combine two lists of Z mass into one
          .filter(lambda event: event.size >= 2) # filter events that has less than two Z
          .map(lambda event: event.pairs(lambda x, y: sqrt(x*x+y*y))) # 2Z->H
          .flatten # pretty result
         )
res = np.array(masses.collect)
res

array([127.17852376, 155.79950464, 107.90775967, 142.96336034,
       153.41439012, 123.86085218, 136.83160538, 128.98982498,
       129.60416187,  96.18799696, 112.39888027, 102.70830056,
       103.47878445,  66.62615023,  48.52350577,  50.13376546,
        75.73577846,  76.77741833,  61.72729286, 142.29445058,
       106.55710517, 123.76143203,  94.02605973,  98.88924908,
       119.9949461 , 135.50373606, 109.01958693, 113.24069062,
        97.30348649,  54.69896098,  62.69098826,  83.39335542,
        88.84050543,  37.63832161, 117.34575125,  98.62094603,
        77.92545522, 125.32304705, 141.40674789,  97.85562869,
       118.14349787,  90.0245886 ,  95.35909069, 123.98768576,
       141.41593695, 136.0850061 , 119.00519265, 134.04816467,
       126.51567219, 120.52749467, 100.84633837, 118.22291883,
       138.392189  , 121.63677474, 136.38980733, 115.39569589,
       130.85427469, 112.98649564, 140.6859679 , 130.25991725,
       142.1552682 ,  95.94705145, 105.25302263, 104.69

In [71]:
import plotly
plotly.tools.set_credentials_file(username='hjhee', api_key='7M7ZO15hQSc0sy01adVG')
plotly.offline.init_notebook_mode(connected=True)

In [99]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np


data = [go.Histogram(x=res)]
py.iplot(data, filename='basic histogram')

# 下面的东西都没有用处，只是试验

In [13]:
masses.take(10)

[[], [], [], [], [179.5715950451616], [], [], [], [], []]

In [22]:
e = events.map(lambda event: [event.electrons.pairs(mass), event.muons.pairs(mass)])

In [25]:
q = e.take(100)
print q
q.map(lambda e: e.zip(lambda x, y: x+y))

[[[], [90.22779776988638]], [[], []], [[], [74.74654928040661]], [[], [89.75736375665977]], [[84.71637776101005], [94.85521728415152]], [[], [92.11672152709482]], [[], [53.36427793158392]], [[], []], [[], [89.84969494859244]], [[], [96.03694205062722]], [[], [86.80058490641416]], [[], [87.29730141854886]], [[], []], [[], []], [[], [94.1401344991302]], [[], [99.82829548826258]], [[], [89.35045512899516]], [[], []], [[], [93.01657608591354]], [[], [87.97456406526169, 128.58445366515267, 62.48648393717207]], [[], [94.41155889760824]], [[], [93.16613921673573]], [[], [89.05260696344838]], [[], []], [[], []], [[], [95.24223125872672]], [[], []], [[], []], [[], [91.1353537369359]], [[], [131.6289837509788]], [[], [84.15809133930381]], [[], []], [[], [121.70734373587467, 93.39859514897753, 22.996373270352922, 62.53167765812187, 42.72818072650716, 44.548414733706565]], [[], []], [[], [29.48155781970828]], [[], []], [[], []], [[], []], [[], [85.6893720090954]], [[], [94.39039849164844]], [[], [

[[],
 [],
 [],
 [],
 [179.5715950451616],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [19]:
events.map(lambda event: event.muons.pairs(mass)).take(20)

[[90.22779776988638],
 [],
 [74.74654928040661],
 [89.75736375665977],
 [94.85521728415152],
 [92.11672152709482],
 [53.36427793158392],
 [],
 [89.84969494859244],
 [96.03694205062722],
 [86.80058490641416],
 [87.29730141854886],
 [],
 [],
 [94.1401344991302],
 [99.82829548826258],
 [89.35045512899516],
 [],
 [93.01657608591354],
 [87.97456406526169, 128.58445366515267, 62.48648393717207]]

In [22]:
mass.__code__

<code object mass at 0x7f8a9d521eb0, file "<ipython-input-12-b16813dd5935>", line 2>


In [15]:
events[19].muons

[<Record at index 33>, <Record at index 34>, <Record at index 35>]

In [32]:
a = [[1], [3], [], [6]]
b = [[1], [3, 9], [7], []]
[a, b].zip(lambda x, y: [x, y].flatten)

[[1, 1], [3, 3, 9], [7], [6]]

In [24]:
c = [[1, 1], [3, 3, 9], [7], [6]]